In [19]:
# Carregar llibreries

import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re
import numpy as np
import random
import time
from google.colab import files

In [20]:
# Carrega el fitxer Excel

# INFORME GPG: Inventari de drets immaterials - Dates de vigència

# nom_fitxer = '/content/Inventaridedretsimmaterials-Datesdevigència (2).xlsx'  # Canvia a la ruta correcta del teu fitxer
# df = pd.read_excel(nom_fitxer, header=13)  # Ajusta el valor de 'header' segons la posició de les capçaleres

# Carrega l'arxiu XLSX
uploaded = files.upload()

# Obtenir el nom de l'arxiu carregat
nom_arxiu = list(uploaded.keys())[0]

# Llegir l'arxiu XLSX amb pandas
df = pd.read_excel(nom_arxiu, header=13)  # Ajusta el valor de 'header' segons la posició de les capçaleres

# Aplica els filtres
df = df[df["Registre d'inscripció"].fillna("") == "Oficina Espanyola de Patents i Marques (OEPM)"]
df = df[df["Estat"] == "Alta"]

# Mostra el DataFrame filtrat
print("Total marques:", len(df))
df.head()

# Selecciona les columnes que necessites
columnes_seleccionades = [
    "Codi registre",
    "Data fi vigència",
    "Data inici vigència",
]

df = df[columnes_seleccionades]


Saving Inventaridedretsimmaterials-Datesdevigència (15).xlsx to Inventaridedretsimmaterials-Datesdevigència (15) (3).xlsx


/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Total marques: 664


In [21]:
df

,Codi registre,Data fi vigència,Data inici vigència
0,3030809,2032-05-18 00:00:00,1992-05-18 00:00:00
1,2070671,2027-01-29 00:00:00,1997-01-29 00:00:00
2,3025317,2032-04-01 00:00:00,2002-04-01 00:00:00
3,2519539,2032-12-27 00:00:00,2002-12-27 00:00:00
12,1316002,2029-04-17 00:00:00,1989-04-17 00:00:00
...,...,...,...
10001,2753757,2027-02-06 00:00:00,2007-02-06 00:00:00
10002,2753765,2027-02-06 00:00:00,2007-02-06 00:00:00
10003,2855924,2028-12-10 00:00:00,2008-12-10 00:00:00
10051,4162453,2032-03-15 00:00:00,2022-03-15 00:00:00


In [22]:
# Filtrar XLSX

# Obtenir la data d'avui
avui = datetime.now()

df2 = df
# print(df2["Codi registre"])

In [23]:
def separar(cadena):
    # Patró de cerca per la data (dd/mm/yyyy)
    patro_data = re.compile(r'\b(\d{1,2}/\d{1,2}/\d{4})\b')

    # Cerca la data a la cadena
    coincidencia_data = patro_data.search(cadena)

    if coincidencia_data:
        # Si es troba la data, separa el text i la data
        text = cadena[:coincidencia_data.start()].strip()
        data = coincidencia_data.group()
    else:
        # Si no es troba la data, retorna la cadena completa com a text i deixa la data buida
        text = cadena
        data = ""

    return text, data

# Exemple d'ús
# cadena = "PUBL.CONCES.RENOV. DE F.RESOL. 06/11/2023"
# text, data = separar(cadena)


In [ ]:
def dades(codi):

    url = "http://consultas2.oepm.es/ceo/jsp/busqueda/consultaExterna.xhtml?numExp=M" + codi

    dades = {}  # Crear un diccionari per emmagatzemar les dades

    dades['Codi'] = codi

    try:
        response = requests.get(url)

        if response.status_code == 200:

          try:

            # Parseja el contingut HTML amb BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')

            # Troba l'element <h4> amb el text "Actos de tramitación:"
            h4_actos = soup.find('h4', string=lambda text: "Actos de tramitación:" in text if text else False)
            h4_actos = h4_actos.find_next('div', class_='datos')

            # Troba tots els elements <td> amb l'atribut role="gridcell"
            td_elements = h4_actos.find_all('td', {'role': 'gridcell'})

            # Verifica si hi ha algun element
            if td_elements:
                # Obtenir l'últim element i extreure'n el text
                dades['last_td'] = td_elements[-1].get_text(strip=True)
                dades['Data publicació'] = td_elements[-2].get_text(strip=True)

                dades['Anotació'], dades['Data anotació'] = separar(dades['last_td'])

            # Busca l'element <h4> que contingui el text "Fecha presentación solicitud otorgada:"
            presentacio_h4 = soup.find('h4', string=lambda text: "Fecha presentación solicitud otorgada:" in text if text else False)

            # Obte el text de l'element següent (en aquest cas, l'element <p>)
            if presentacio_h4:
                dades['Data de presentació'] = presentacio_h4.find_next('p').get_text(strip=True)

            # Busca l'element <h4> que contingui el text "Estado:"
            estado_h4 = soup.find('h4', string=lambda text: "Estado:" in text if text else False)

            # Obte el text de l'element següent (en aquest cas, l'element <p>)
            if estado_h4:
                dades['Estat'] = estado_h4.find_next('p').get_text(strip=True)

            # Troba l'element <div> amb l'id "marco-imagen-sitamar"
            div_marco_imagen = soup.find('div', {'id': 'marco-imagen-sitamar'})

            if div_marco_imagen:
                # Troba l'element <img> dins del div
                img_element = div_marco_imagen.find('img')

                if img_element:
                    # Obté el valor de l'atribut src
                    src_valor = img_element.get('src', '')

                    # Imprimeix el resultat
                    dades['Imatge'] = "http://consultas2.oepm.es"+src_valor
                else:
                    dades['Imatge'] = ""

            # Troba l'element <h3>
            h3_element = soup.find('h3')

            if h3_element:
                # Obté el text de l'element <h3>
                text_h3 = h3_element.get_text(strip=True)

                # Utilitza una expressió regular per seleccionar el segon grup
                match = re.match(r'(.*)\s-\s(.*)', text_h3)

                if match:
                    segon_grup = match.group(2)
                    dades['Denominació'] = segon_grup

            # Troba l'element <h4>
            h4_element = soup.find('h4', string=lambda text: "Tipo:" in text if text else False)

            if h4_element:
                # Obtenir el text de l'element <h4>
                text_h4 = h4_element.find_next('p').get_text(strip=True)

                dades['Tipus'] = text_h4

            # Extreu el text que segueix a <span class="campo">Nombre:</span>
            nom_span = soup.find('span', class_='campo', string="Nombre:")
            if nom_span:
                text_nom = nom_span.find_next('div', class_='datos-sitamar-divSubRight').text.strip()
                # Comprova si hi ha l'etiqueta <br/>
                if "\n" in text_nom:
                    # Divideix el text en dos blocs amb l'etiqueta <br />
                    motiu, titular = map(str.strip, text_nom.split('\n', 1))
                    dades['Titular OEPM'] = titular
                    dades['Motiu'] = motiu
                else:
                    # Si no hi ha <br />, posa tot el text a 'titular'
                    dades['Titular OEPM'] = text_nom
                    dades['Motiu'] = ""

            dades['Url'] = url
          except:
              dades['Estat'] = "Núm. registre incorrecte"

        else:
            print(f"S'ha produït un error en la sol·licitud. Codi d'estat: {response.status_code}")

    except requests.RequestException as e:
        print(f"Error de connexió: {e}")

    return dades  # Retorna el diccionari amb les dades

In [ ]:
'''
# Exemple d'ús

resultats = dades("2551345")

# Ara pots fer el que vulguis amb les dades obtingudes
if resultats:
    # Fer quelcom amb les dades, com ara imprimir-les
    print("Dades obtingudes:")
    print(f"codi: {resultats.get('Codi')}")
    print(f"denominació: {resultats.get('Denominació')}")
    print(f"estado_text: {resultats.get('Estat')}")
    print(f"data_publicacio: {resultats.get('Data publicació')}")
    print(f"anotacio: {resultats.get('Anotació')}")
    print(f"anotacio_data: {resultats.get('Data anotació')}")
    print(f"imatge: {resultats.get('Imatge')}")
    print(f"titular: {resultats.get('Titular OEPM')}")
    print(f"motiu: {resultats.get('Motiu')}")
    print(f"tipus: {resultats.get('Tipus')}")
    print(f"Url: {resultats.get('Url')}")
'''

'\n# Exemple d\'ús\n\nresultats = dades("2551345")\n\n# Ara pots fer el que vulguis amb les dades obtingudes\nif resultats:\n    # Fer quelcom amb les dades, com ara imprimir-les\n    print("Dades obtingudes:")\n    print(f"codi: {resultats.get(\'Codi\')}")\n    print(f"denominació: {resultats.get(\'Denominació\')}")\n    print(f"estado_text: {resultats.get(\'Estat\')}")\n    print(f"data_publicacio: {resultats.get(\'Data publicació\')}")\n    print(f"anotacio: {resultats.get(\'Anotació\')}")\n    print(f"anotacio_data: {resultats.get(\'Data anotació\')}")\n    print(f"imatge: {resultats.get(\'Imatge\')}")\n    print(f"titular: {resultats.get(\'Titular OEPM\')}")\n    print(f"motiu: {resultats.get(\'Motiu\')}")\n    print(f"tipus: {resultats.get(\'Tipus\')}")\n    print(f"Url: {resultats.get(\'Url\')}")\n'

In [ ]:
# Baixar la informació

marques_revisades = pd.DataFrame()

id = df2["Codi registre"]

print("Marques a processar: ", len(id), "\n")

for index, x in enumerate(id):
    print(str(round(index/len(id)*100, 2))+"%")
    resultats = dades(str(x))
    marques_revisades = pd.concat([marques_revisades, pd.DataFrame([resultats])], ignore_index=True)
    time.sleep(random.uniform(10, 15))
print("100.00 %")

marques_revisades.drop('last_td', axis=1, inplace=True)
marques_revisades

Marques a processar:  3 

0.0%
33.33%
66.67%
100.00 %


,Codi,Data publicació,Anotació,Data anotació,Data de presentació,Estat,Imatge,Denominació,Tipus,Titular OEPM,Motiu,Url
0,3030809,01/09/2022,PUBL.CONCES.RENOV. DE F.RESOL.,25/08/2022,18/05/1992,En vigor,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,GENERALITAT DE CATALUNYA,Mixta,GENERALITAT DE CATALUNYA,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...
1,2070671,22/03/2017,PUBL.CONCES.RENOV. DE F.RESOL.,15/03/2017,29/01/1997,En vigor,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,GENERALITAT DE CATALUNYA,Mixta,GENERALITAT DE CATALUNYA,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...
2,3025317,14/07/2022,PUBL.CONCES.RENOV. DE F.RESOL.,07/07/2022,01/04/2002,En vigor,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,GENERALITAT DE CATALUNYA,Mixta,GENERALITAT DE CATALUNYA,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...


In [ ]:
import pandas as pd

# Defineix l'ordre desitjat de les columnes
ordre_columnes = ['Codi', 'Denominació', 'Tipus', 'Estat', 'Data de presentació', 'Data publicació', 'Anotació', 'Data anotació', 'Titular OEPM', 'Motiu', 'Imatge', 'Url']

# Reordena les columnes del DataFrame original
marques_revisades = marques_revisades[ordre_columnes]
marques_revisades = marques_revisades.fillna("")

In [ ]:
marques_revisades.to_excel(f'marques_oepm_extracció_{avui.year}{avui.month:02d}{avui.day:02d}.xlsx', index=False)

In [ ]:
files.download(f'marques_oepm_extracció_{avui.year}{avui.month:02d}{avui.day:02d}.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>